tenemos que comprobar que de cada drugs - cell lines hay informacion de las firmas genicas en cmaps 

## CMAPS

(chat)

El Connectivity Map (CMAP) de Broad Institute es una gran base de datos de perfiles de expresión génica de células tratadas con distintos compuestos químicos, ARN interferente, etc.

El archivo GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328.gctx contiene los perfiles de expresión a nivel 5, que son perfiles de consenso (e.g. usando el algoritmo MODZ) de muchos experimentos.



Me he descargado los datos de https://www.ncbi.nlm.nih.gov/geo/query/acc.cgi?acc=GSE70138 
* GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx.gz : level 5 components
* GSE70138_Broad_LINCS_pert_info.txt.gz	: pert info
* GSE70138_Broad_LINCS_cell_info_2017-04-28.txt.gz : cell info 
* GSE70138_Broad_LINCS_gene_info_2017-03-06.txt.gz : gene info
* GSE70138_Broad_LINCS_inst_info_2017-03-06.txt.gz	: inst into
* GSE70138_Broad_LINCS_sig_info_2017-03-06.txt.gz	: sig info
* GSE70138_Broad_LINCS_sig_metrics_2017-03-06.txt.gz	 : sig metrics 



In [40]:
# paquete que se utiliza para procesar los datos
!pip install cmapPy

  Using cached h5py-3.13.0-cp39-cp39-win_amd64.whl.metadata (2.5 kB)
Using cached h5py-3.13.0-cp39-cp39-win_amd64.whl (3.0 MB)


In [1]:
import pandas as pd
import gzip, shutil
import os
import pkg_resources

from cmapPy.pandasGEXpress.parse import parse

C:\Users\Usuario\AppData\Local\Temp\ipykernel_18308\2811086252.py:4: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources


**1) Abrimos metadatos**

In [2]:
import pandas as pd
import gzip

# gene info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_gene_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    gene_info = pd.read_csv(f, sep='\t')

# cell info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_cell_info_2017-04-28.txt.gz"
with gzip.open(file_path, 'rt') as f:
    cell_info = pd.read_csv(f, sep='\t')

# pert info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_pert_info.txt.gz"
with gzip.open(file_path, 'rt') as f:
    pert_info = pd.read_csv(f, sep='\t')

# inst info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_inst_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    inst_info = pd.read_csv(f, sep='\t')

# sig info 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_sig_info_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    sig_info = pd.read_csv(f, sep='\t')

# sig metrics 
file_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_sig_metrics_2017-03-06.txt.gz"
with gzip.open(file_path, 'rt') as f:
    sig_metrics = pd.read_csv(f, sep='\t')


exploramos un poco los metadatos

In [3]:
gene_info.head()

,pr_gene_id,pr_gene_symbol,pr_gene_title,pr_is_lm,pr_is_bing
0,780,DDR1,discoidin domain receptor tyrosine kinase 1,1,1
1,7849,PAX8,paired box 8,1,1
2,2978,GUCA1A,guanylate cyclase activator 1A,0,0
3,2049,EPHB3,EPH receptor B3,0,1
4,2101,ESRRA,estrogen related receptor alpha,0,1


In [4]:
cell_info.head()

,cell_id,cell_type,base_cell_id,precursor_cell_id,modification,sample_type,primary_site,subtype,original_growth_pattern,provider_catalog_id,original_source_vendor,donor_age,donor_sex,donor_ethnicity
0,A375,cell line,A375,-666,-666,tumor,skin,malignant melanoma,adherent,CRL-1619,ATCC,54,F,-666
1,A375.311,cell line,A375,A375,genetically modified to stably express Cas9 pr...,tumor,skin,malignant melanoma,adherent,CRL-1619,ATCC,54,F,-666
2,A549,cell line,A549,-666,-666,tumor,lung,non small cell lung cancer| carcinoma,adherent,CCL-185,ATCC,58,M,Caucasian
3,A549.311,cell line,A549,A549,genetically modified to stably express Cas9 p...,tumor,lung,non small cell lung cancer| carcinoma,adherent,CCL-185,ATCC,58,M,Caucasian
4,A673,cell line,A673,-666,-666,tumor,bone,ewing's sarcoma,adherent,CRL-1598,ATCC,-666,F,-666


In [5]:
cell_info[cell_info["cell_type"] == "cell line"]["subtype"].unique()

array(['malignant melanoma', 'non small cell lung cancer| carcinoma',
       "ewing's sarcoma", 'adenocarcinoma', 'carcinoma',
       'colorectal adenocarcinoma',
       'non small cell lung cancer| large cell carcinoma',
       'carcinoma| epithelial-mucinous',
       'non small cell lung cancer| adenocarcinoma',
       'mucinous papillary adenocarcinoma', 'normal kidney',
       'non small cell lung cancer| squamous cell carcinoma',
       'colorectal carcinoma', 'endometrial adenocarcinoma',
       'embryonal kidney', 'epithelial', 'hepatocellular carcinoma',
       'acute myelogenous leukemia (AML)| M3 (promyelocytic)', '-666',
       'bone marrow', 'normal endothelial cell|umbilical cord',
       'endometrial adenocarcinoma| endometrioid carcinoma',
       'acute lymphoblastic leukemia (ALL)| T-cell',
       'carcinoma| prostate', 'skin fibroblast',
       'small cell lung cancer| carcinoma',
       'non small cell lung cancer| adenosquamous carcinoma',
       'kidney epithelial',

In [6]:
cell_info['cell_id'].unique()

array(['A375', 'A375.311', 'A549', 'A549.311', 'A673', 'AGS', 'BT20',
       'CL34', 'CORL23', 'COV644', 'DV90', 'EFO27', 'H1299', 'HA1E',
       'HA1E.101', 'HA1E.311', 'HCC15', 'HCC515', 'HCT116', 'HEC108',
       'HEK293T', 'HEKTE', 'HELA', 'HELA.311', 'HEPG2', 'HL60', 'HME1',
       'HS27A', 'HS578T', 'HT115', 'HT29', 'HT29.311', 'HUH7', 'HUVEC',
       'JHUEM2', 'JURKAT', 'LNCAP', 'LOVO', 'MCF10A', 'MCF7', 'MCF7.101',
       'MCF7.311', 'MCH58', 'MDAMB231', 'MDST8', 'NCIH1694', 'NCIH1836',
       'NCIH2073', 'NCIH508', 'NCIH596', 'NCIH716', 'NKDBA', 'NOMO1',
       'OV7', 'PC3', 'PC3.101', 'PC3.311', 'PL21', 'RKO', 'RMGI', 'RMUGS',
       'SHSY5Y', 'SKBR3', 'SKLU1', 'SKM1', 'SKMEL1', 'SKMEL28', 'SNGM',
       'SNU1040', 'SNUC5', 'SW480', 'SW620', 'SW948', 'T3M10', 'THP1',
       'TYKNU', 'U266', 'U2OS', 'U937', 'VCAP', 'WSUDLCL2', 'YAPC',
       'YAPC.311', 'MNEU.E', 'NEU', 'NEU.KCL', 'NPC', 'NPC.CAS9',
       'NPC.TAK', 'HUES3', 'FIBRNPC', 'ASC', 'ASC.C', 'CD34', 'PHH',
       'S

hay informacion de neuroblastoma pero solo esta la linea celular 

In [7]:
cell_info[(cell_info['cell_type'] == 'cell line') & (cell_info['subtype'] == 'neuroblastoma')]

,cell_id,cell_type,base_cell_id,precursor_cell_id,modification,sample_type,primary_site,subtype,original_growth_pattern,provider_catalog_id,original_source_vendor,donor_age,donor_sex,donor_ethnicity
61,SHSY5Y,cell line,SHSY5Y,-666,-666,tumor,autonomic ganglia,neuroblastoma,mix,CRL-2266,ATCC,-666,F,-666


In [8]:
pert_info.head() ##son las drogas

,pert_id,canonical_smiles,inchi_key,pert_iname,pert_type
0,BRD-K70792160,CCN(CC)CCCCN1c2ccccc2Oc2ccc(Cl)cc12,GYBXAGDWMCJZJK-UHFFFAOYSA-N,10-DEBC,trt_cp
1,BRD-K68552125,CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...,PHEDXBVPIONUQT-RGYGYFBISA-N,phorbol-myristate-acetate,trt_cp
2,BRD-K92301463,CCCCC(C)(C)[C@H](O)\C=C\[C@H]1[C@H](O)CC(=O)[C...,QAOBBBBDJSWHMU-WMBBNPMCSA-N,"16,16-dimethylprostaglandin-e2",trt_cp
3,BRD-A29731977,CCCCCC(=O)O[C@@]1(CCC2C3CCC4=CC(=O)CC[C@]4(C)C...,DOMWKUIIPQCAJU-JKPPDDDBSA-N,17-hydroxyprogesterone-caproate,trt_cp
4,BRD-K07954936,OC(=O)CCCC[C@@H]1SC[C@@H]2NC(=N)N[C@H]12,WWVANQJRLPIHNS-ZKWXMUAHSA-N,2-iminobiotin,trt_cp


In [9]:
pert_info[pert_info['pert_iname'] == 'PD-0325901']

,pert_id,canonical_smiles,inchi_key,pert_iname,pert_type
1209,BRD-K49865102,OC[C@@H](O)CONC(=O)c1ccc(F)c(F)c1Nc1ccc(I)cc1F,SUDAHWBOROXANE-SECBINFHSA-N,PD-0325901,trt_cp


In [10]:
pert_info['pert_type'].unique() ##hay qye quedarse con cp que es component 
pert_cp = pert_info[pert_info['pert_type'] == 'trt_cp'] ##component es drug 

In [11]:
pert_cp.shape[0] ## 1796 drugs solo ?? 

1796

In [12]:
pert_cp

,pert_id,canonical_smiles,inchi_key,pert_iname,pert_type
0,BRD-K70792160,CCN(CC)CCCCN1c2ccccc2Oc2ccc(Cl)cc12,GYBXAGDWMCJZJK-UHFFFAOYSA-N,10-DEBC,trt_cp
1,BRD-K68552125,CCCCCCCCCCCCCC(=O)O[C@@H]1[C@@H](C)[C@]2(O)[C@...,PHEDXBVPIONUQT-RGYGYFBISA-N,phorbol-myristate-acetate,trt_cp
2,BRD-K92301463,CCCCC(C)(C)[C@H](O)\C=C\[C@H]1[C@H](O)CC(=O)[C...,QAOBBBBDJSWHMU-WMBBNPMCSA-N,"16,16-dimethylprostaglandin-e2",trt_cp
3,BRD-A29731977,CCCCCC(=O)O[C@@]1(CCC2C3CCC4=CC(=O)CC[C@]4(C)C...,DOMWKUIIPQCAJU-JKPPDDDBSA-N,17-hydroxyprogesterone-caproate,trt_cp
4,BRD-K07954936,OC(=O)CCCC[C@@H]1SC[C@@H]2NC(=N)N[C@H]12,WWVANQJRLPIHNS-ZKWXMUAHSA-N,2-iminobiotin,trt_cp
...,...,...,...,...,...
1792,BRD-K86787118,CO[C@H]1\C=C\C=C(C)\C[C@@H](C)[C@H](O)[C@H](C)...,DPMRXCGDQWBYHC-ZOMSYMKESA-N,bafilomycin a1,trt_cp
1793,BRD-K87465484,COc1cc2c(NC3CCN(CC3)C3CCCCC3)nc(nc2cc1OCCCN1CC...,OUKWLRHRXOPODD-UHFFFAOYSA-N,unc-0646,trt_cp
1794,BRD-K90860366,CCCc1cc2c(ncnc2s1)N1CCN(CC1)C1=NCC(C)(C)S1,SRQYLNYQAPCPIR-UHFFFAOYSA-N,1271738-62-5,trt_cp
1795,BRD-K92123432,COc1cc(O)c2c(c1)oc(cc2=O)-c1ccc(OC)c(c1)-c1c(O...,AIFCFBUSLAEIBR-UHFFFAOYSA-N,ginkgetin,trt_cp


In [13]:
inst_info.head()

,inst_id,cell_id,det_plate,det_well,pert_mfc_id,pert_dose,pert_dose_unit,pert_id,pert_iname,pert_type,pert_time,pert_time_unit
0,LJP005_A375_24H_X1_B19:A03,A375,LJP005_A375_24H_X1_B19,A03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,24.0,h
1,LPROT001_A375_6H_X1_B20:B03,A375,LPROT001_A375_6H_X1_B20,B03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h
2,LPROT001_A375_6H_X1_B20:B05,A375,LPROT001_A375_6H_X1_B20,B05,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h
3,LPROT002_A375_6H_X1_B22:B03,A375,LPROT002_A375_6H_X1_B22,B03,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h
4,LPROT002_A375_6H_X1_B22:B05,A375,LPROT002_A375_6H_X1_B22,B05,DMSO,-666.0,-666,DMSO,DMSO,ctl_vehicle,6.0,h


In [14]:
sig_info.head()

,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
0,LJP005_A375_24H:A03,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A03|LJP005_A375_24H_X2_...
1,LJP005_A375_24H:A04,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A04|LJP005_A375_24H_X2_...
2,LJP005_A375_24H:A05,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A05|LJP005_A375_24H_X2_...
3,LJP005_A375_24H:A06,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A06|LJP005_A375_24H_X2_...
4,LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...


In [15]:
sig_metrics.head()

,Unnamed: 0,sig_id,distil_cc_q75,distil_ss,pert_id,pert_iname,pert_type,tas,ngenes_modulated_up_lm,ngenes_modulated_dn_lm,pct_self_rank_q25,distil_nsample
0,0,REP.A001_A375_24H:A03,0.10,4.11955,DMSO,DMSO,ctl_vehicle,0.131454,36,28,18.437500,3
1,1,REP.A001_A375_24H:A04,0.10,3.47207,DMSO,DMSO,ctl_vehicle,0.105571,26,20,12.924107,3
2,2,REP.A001_A375_24H:A05,0.10,3.01546,DMSO,DMSO,ctl_vehicle,0.100102,17,16,12.120536,3
3,3,REP.A001_A375_24H:A06,0.10,3.93260,DMSO,DMSO,ctl_vehicle,0.120920,30,24,16.718750,3
4,4,REP.A001_A375_24H:A07,0.12,3.02548,BRD-K25114078,aminoguanidine,trt_cp,0.111322,18,12,11.629464,3


In [16]:
#####
sig_info[sig_info['cell_id'] == 'A549']

,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
379,LJP005_A549_24H:A03,DMSO,DMSO,ctl_vehicle,A549,-666,24 h,LJP005_A549_24H_X1_B19:A03|LJP005_A549_24H_X2_...
380,LJP005_A549_24H:A04,DMSO,DMSO,ctl_vehicle,A549,-666,24 h,LJP005_A549_24H_X1_B19:A04|LJP005_A549_24H_X2_...
381,LJP005_A549_24H:A05,DMSO,DMSO,ctl_vehicle,A549,-666,24 h,LJP005_A549_24H_X1_B19:A05|LJP005_A549_24H_X2_...
382,LJP005_A549_24H:A06,DMSO,DMSO,ctl_vehicle,A549,-666,24 h,LJP005_A549_24H_X1_B19:A06|LJP005_A549_24H_X2_...
383,LJP005_A549_24H:A07,BRD-K76908866,CP-724714,trt_cp,A549,10.0 um,24 h,LJP005_A549_24H_X1_B19:A07|LJP005_A549_24H_X2_...
...,...,...,...,...,...,...,...,...
39388,LPROT003_A549_6H:O16,BRD-A13254067,okadaic acid,trt_cp,A549,1.0 um,6 h,LPROT003_A549_6H_X1.A2_B22:O16
39389,LPROT003_A549_6H:O18,BRD-A13254067,okadaic acid,trt_cp,A549,1.0 um,6 h,LPROT003_A549_6H_X1.A2_B22:O18
39390,LPROT003_A549_6H:O20,BRD-K16189898,CHIR-99021,trt_cp,A549,2.0 um,6 h,LPROT003_A549_6H_X1.A2_B22:O20
39391,LPROT003_A549_6H:O22,BRD-K16189898,CHIR-99021,trt_cp,A549,2.0 um,6 h,LPROT003_A549_6H_X1.A2_B22:O22


In [17]:
sig_info.head()

,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
0,LJP005_A375_24H:A03,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A03|LJP005_A375_24H_X2_...
1,LJP005_A375_24H:A04,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A04|LJP005_A375_24H_X2_...
2,LJP005_A375_24H:A05,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A05|LJP005_A375_24H_X2_...
3,LJP005_A375_24H:A06,DMSO,DMSO,ctl_vehicle,A375,-666,24 h,LJP005_A375_24H_X1_B19:A06|LJP005_A375_24H_X2_...
4,LJP005_A375_24H:A07,BRD-K76908866,CP-724714,trt_cp,A375,10.0 um,24 h,LJP005_A375_24H_X1_B19:A07|LJP005_A375_24H_X2_...


**Haacemos intento de parsear**

In [18]:
# get all samples (across all cell types, doses, and other treatment conditions) that were treated with aspirin
aspirin_ids = sig_info["sig_id"][sig_info["pert_iname"] == "fluorouracil"]
# how many samples are there in this data set?
print("number of samples treated with aspirin:")
len(aspirin_ids)

number of samples treated with aspirin:


42

In [19]:
## Just once
#import gzip
#import shutil

# Ruta original y destino
#input_path = r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx.gz"
#output_path = input_path[:-3]  # Quita el .gz

# Descomprimir
#with gzip.open(input_path, 'rb') as f_in:
#    with open(output_path, 'wb') as f_out:
#        shutil.copyfileobj(f_in, f_out)


In [20]:
# con esto saco solo aquellas firmas cuyas columnas sean las firmas del farmaco que he seleccionado antes (en el ejemplo, aspirina)
level5_lincs_subset = parse(r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\GSE70138_Broad_LINCS_Level5_COMPZ_n118050x12328_2017-03-06.gctx", cid=aspirin_ids)


C:\Users\Usuario\anaconda3\envs\myenv\lib\site-packages\cmapPy\pandasGEXpress\parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))
C:\Users\Usuario\anaconda3\envs\myenv\lib\site-packages\cmapPy\pandasGEXpress\parse_gctx.py:275: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  meta_df = meta_df.apply(lambda x: pd.to_numeric(x, errors="ignore"))


**data_df** contiene un DataFrame de expresión génica, donde:

* Las filas (rid) representan genes (identificados por su número o ID).
* Las columnas (cid) son las firmas tratadas con aspirina (cada una con un sig_id).
* Los valores son scores de expresión génica (e.g., Z-scores) después del tratamiento.

**Firmas génicas** Una firma en este contexto es un perfil de expresión génica que resume cómo cambió la expresión de muchos genes en una célula después de ser tratada con un compuesto (como aspirina). Es decir, representa el efecto del tratamiento sobre el transcriptoma de una célula.

Eg REP.A004_A375_24H:J19
* REP	Tipo de firma: replicate-level (una repetición experimental concreta). También existen MODZ (consenso entre replicados) o CTR (control).
* A004	ID del batch o placa (Plate A004).
* A375	Nombre de la línea celular usada (A375 = melanoma humano).
* 24H	Tiempo de exposición al compuesto: 24 horas.
* J19	Posición del pozo en la placa (fila J, columna 19). Cada pozo es una muestra tratada diferente.

Eg REP.A004_HA1E_24H:J19

* Mismo batch y tiempo (A004, 24H)
* Línea celular: HA1E (una línea epitelial renal humana inmortalizada)
* Pozo J19
* También una repetición individual

In [21]:
level5_lincs_subset.data_df

cid,REP.A004_A375_24H:J19,REP.A004_A375_24H:J20,REP.A004_A375_24H:J21,REP.A004_A375_24H:J22,REP.A004_A375_24H:J23,REP.A004_A375_24H:J24,REP.A004_HA1E_24H:J19,REP.A004_HA1E_24H:J20,REP.A004_HA1E_24H:J21,REP.A004_HA1E_24H:J22,...,REP.A004_PC3_24H:J21,REP.A004_PC3_24H:J22,REP.A004_PC3_24H:J23,REP.A004_PC3_24H:J24,REP.A004_YAPC_24H:J19,REP.A004_YAPC_24H:J20,REP.A004_YAPC_24H:J21,REP.A004_YAPC_24H:J22,REP.A004_YAPC_24H:J23,REP.A004_YAPC_24H:J24
rid,,,,,,,,,,,,,,,,,,,,,
780,-0.399637,-0.770754,0.189234,-1.143714,-0.327838,-0.271492,-0.480273,-0.315523,-0.389883,-0.832531,...,-0.200128,0.544727,0.541837,0.795481,-0.212451,0.511522,-0.299184,0.240996,-0.51980,0.032531
7849,0.280725,-0.014461,0.308759,0.385202,-1.415056,0.087253,0.382977,0.068801,-0.307338,0.339158,...,-0.713788,0.949827,0.263056,-1.295408,0.266813,-0.167321,-0.279765,0.663566,0.29355,-0.391949
2978,0.532132,-0.567018,0.207542,0.990786,0.244507,-0.427288,-0.703153,-0.263904,-0.671844,-1.336173,...,-1.332174,-0.174949,0.024814,-0.446468,-0.556293,-0.605076,-1.128049,-0.510255,-0.63730,-0.162908
2049,1.028017,0.195197,-0.886888,0.133362,0.497969,-0.071813,-0.679146,1.662045,0.338156,1.594850,...,-0.308859,-0.745074,-0.977798,0.459082,-0.427482,-0.949891,-0.604301,-1.626754,-0.04085,1.252244
2101,0.763196,0.858211,-0.681884,0.123427,-0.464446,1.274661,-0.535858,-0.838806,0.409727,-0.329192,...,0.272278,-0.160282,-0.224858,1.499270,-1.765094,-1.376031,0.577341,0.485813,-1.33850,0.649889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4034,0.824345,0.008302,1.485423,0.558783,0.848364,-0.445777,-0.502469,-0.617494,0.446174,-0.044188,...,0.896303,-0.008991,-0.175890,0.455030,-1.051623,-1.308150,-0.415748,-0.678362,0.24480,-0.551496
399664,-1.068511,0.365703,-0.728701,0.375528,-0.258713,0.583257,-0.581598,0.069967,0.049532,-0.854778,...,-0.000891,0.452721,0.329818,-0.150849,-0.414956,-0.542207,-0.491885,-0.730318,-0.01295,0.183141
54869,-1.349467,-0.276604,-0.357715,-0.840061,-0.159780,-0.986922,-1.122096,-1.271474,-0.822650,-1.363488,...,-0.243153,-1.002177,0.368592,-0.226475,-0.881396,0.426503,0.155209,-0.483945,1.26415,0.059384


In [22]:
level5_lincs_subset.col_metadata_df.head(10)


chd
cid
REP.A004_A375_24H:J19
REP.A004_A375_24H:J20
REP.A004_A375_24H:J21
REP.A004_A375_24H:J22
REP.A004_A375_24H:J23
REP.A004_A375_24H:J24
REP.A004_HA1E_24H:J19
REP.A004_HA1E_24H:J20
REP.A004_HA1E_24H:J21


**sig_id meaning:**
- **REP**: Indicates a replicate-level signature (as opposed to a control or consensus like MODZ).
- **A005**: Plate or batch identifier (e.g., Plate A005).
- **A375**: Cell line used in the experiment (here, A375 melanoma cells).
- **24H**: Duration of treatment (24 hours).
- **:O19**: Well position on the plate (row O, column 19).


In [23]:
# Obtener una lista de todos los identificadores de columna (cid)
cids = level5_lincs_subset.col_metadata_df.index.tolist()

# Filtrar los cids que contienen 'A375'
columnas_A375 = [cid for cid in cids if 'A375' in cid]

# Verificar cuántas columnas se encontraron
print(f"Se encontraron {len(columnas_A375)} columnas asociadas con la línea celular 'A375'.")


Se encontraron 6 columnas asociadas con la línea celular 'A375'.


In [24]:
# Seleccionar las columnas filtradas en data_df
datos_A375 = level5_lincs_subset.data_df[columnas_A375]

# Mostrar las primeras filas de los datos seleccionados
datos_A375.head()

cid,REP.A004_A375_24H:J19,REP.A004_A375_24H:J20,REP.A004_A375_24H:J21,REP.A004_A375_24H:J22,REP.A004_A375_24H:J23,REP.A004_A375_24H:J24
rid,,,,,,
780,-0.399637,-0.770754,0.189234,-1.143714,-0.327838,-0.271492
7849,0.280725,-0.014461,0.308759,0.385202,-1.415056,0.087253
2978,0.532132,-0.567018,0.207542,0.990786,0.244507,-0.427288
2049,1.028017,0.195197,-0.886888,0.133362,0.497969,-0.071813
2101,0.763196,0.858211,-0.681884,0.123427,-0.464446,1.274661


**Juntamos con drugcomb**

Sacamos las drogas con las lineas celulares a las que están asociadas

In [25]:
summary_drugcomb=pd.read_csv(r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\combinations_cmaps_drugcomb.csv")

In [26]:
summary_drugcomb.columns

Index(['block_id', 'drug_row_id', 'drug_col_id', 'cell_line_id', 'conc_r_unit',
       'conc_c_unit', 'synergy_zip', 'synergy_loewe', 'synergy_hsa',
       'synergy_bliss', 'ic50_row', 'ic50_col', 'ri_row', 'ri_col', 'css_row',
       'css_col', 'css_ri', 'S_sum', 'quality', 'S_mean', 'S_max', 'dname_row',
       'chembl_id_row', 'inchikey_row', 'smiles_row', 'molecular_formula_row',
       'drugbank_id_row', 'kegg_id_row', 'cmaps_name_row', 'dname_col',
       'chembl_id_col', 'inchikey_col', 'smiles_col', 'molecular_formula_col',
       'drugbank_id_col', 'kegg_id_col', 'cmaps_name_col', 'cell_line_name',
       'cell_disease_id', 'tumor_location_cosmic',
       'tumor_location_cellosaurus', 'disease_cui'],
      dtype='object')

In [27]:
summary_drugcomb[['cmaps_name_row', 'cmaps_name_col', 'cell_line_name']].head(2)


,cmaps_name_row,cmaps_name_col,cell_line_name
0,fluorouracil,veliparib,A375
1,fluorouracil,MK-1775,A375


make a dicc to collect al the drugs in drugcomb and the cell lines they are used in

In [28]:
from collections import defaultdict

# Initialize dictionary with sets to avoid duplicates
drug_to_cell_lines = defaultdict(set)

# Iterate through each row
for _, row in summary_drugcomb.iterrows():
    drug1 = row['cmaps_name_row']
    drug2 = row['cmaps_name_col']
    cell = row['cell_line_name']
    
    drug_to_cell_lines[drug1].add(cell)
    drug_to_cell_lines[drug2].add(cell)

# Convert sets to lists
drug_to_cell_lines = {drug: list(cells) for drug, cells in drug_to_cell_lines.items()}


In [29]:
len(drug_to_cell_lines) ## there are 49 unique drugs the dictionary is okay 

49

ahora vamos iterando el diccionario para:
* Iterar por cada droga en tu diccionario (drug_to_cell_lines).
* Para cada droga, ver en sig_info cuántas firmas (sig_id) hay asociadas a cada una de sus líneas celulares.
* Guardar o imprimir ese conteo.

In [30]:
import pandas as pd

def count_signatures_per_drug_and_cellline(drug_to_cell_lines, sig_info):
    data = []

    for drug, cell_lines in drug_to_cell_lines.items():
        for cell in cell_lines:
            matching_sigs = sig_info[
                (sig_info["pert_iname"] == drug) &
                (sig_info["cell_id"] == cell)
            ]
            num_signatures = len(matching_sigs)
            data.append({
                "drug": drug,
                "cell_line": cell,
                "num_signatures": num_signatures
            })

    return pd.DataFrame(data)


In [31]:
signatures_df = count_signatures_per_drug_and_cellline(drug_to_cell_lines, sig_info)

In [32]:
signatures_df

,drug,cell_line,num_signatures
0,fluorouracil,A375,6
1,fluorouracil,LOVO,0
2,fluorouracil,HCT116,0
3,fluorouracil,A549,0
4,fluorouracil,MCF7,6
...,...,...,...
272,vismodegib,HCT116,0
273,vandetanib,A549,6
274,vandetanib,MCF7,12
275,vandetanib,HT29,12


In [33]:
import pandas as pd

def count_and_list_signatures_per_drug_and_cellline(drug_to_cell_lines, sig_info):
    data = []

    for drug, cell_lines in drug_to_cell_lines.items():
        for cell in cell_lines:
            matching_sigs = sig_info[
                (sig_info["pert_iname"] == drug) &
                (sig_info["cell_id"] == cell)
            ]
            sig_ids = matching_sigs["sig_id"].unique().tolist()
            num_signatures = len(sig_ids)
            data.append({
                "drug": drug,
                "cell_line": cell,
                "num_signatures": num_signatures,
                "signature_ids": sig_ids
            })

    return pd.DataFrame(data)


In [34]:
drugs_cell_signature_names = count_and_list_signatures_per_drug_and_cellline(drug_to_cell_lines, sig_info)


In [35]:
drugs_cell_signature_names

,drug,cell_line,num_signatures,signature_ids
0,fluorouracil,A375,6,"[REP.A004_A375_24H:J19, REP.A004_A375_24H:J20,..."
1,fluorouracil,LOVO,0,[]
2,fluorouracil,HCT116,0,[]
3,fluorouracil,A549,0,[]
4,fluorouracil,MCF7,6,"[REP.A004_MCF7_24H:J19, REP.A004_MCF7_24H:J20,..."
...,...,...,...,...
272,vismodegib,HCT116,0,[]
273,vandetanib,A549,6,"[LJP007_A549_24H:P13, LJP007_A549_24H:P14, LJP..."
274,vandetanib,MCF7,12,"[LJP007_MCF7_24H:P13, LJP007_MCF7_24H:P14, LJP..."
275,vandetanib,HT29,12,"[LJP007_HT29_24H:P13, LJP007_HT29_24H:P14, LJP..."


para ir probando ver las firmas para x droga y linea celular

In [36]:
# Filtrar por el fármaco y la línea celular específicos
filtered_row = drugs_cell_signature_names[
    (drugs_cell_signature_names["drug"] == "vandetanib") &
    (drugs_cell_signature_names["cell_line"] == "HT29")
]

# Extraer la lista de firmas génicas (signature_ids)
signature_list = filtered_row["signature_ids"].values[0]

A sig_id represents a single, consensus signature — a processed and summarized result of one perturbation experiment. It is generated by aggregating multiple replicates, each of which has its own distil_id. You can think of sig_id as a "final profile" for a specific condition (compound + dose + cell line + timepoint).



In [37]:
# Filtrar sig_info por los sig_id contenidos en la lista
matching_sig_info = sig_info[sig_info["sig_id"].isin(signature_list)]
# Filtrar sig_info por los sig_id contenidos en la lista
matching_sig_info 


,sig_id,pert_id,pert_iname,pert_type,cell_id,pert_idose,pert_itime,distil_id
19691,LJP007_HT29_24H:P13,BRD-K77625799,vandetanib,trt_cp,HT29,10.0 um,24 h,LJP007_HT29_24H_X1_B20:P13|LJP007_HT29_24H_X2_...
19692,LJP007_HT29_24H:P14,BRD-K77625799,vandetanib,trt_cp,HT29,3.33 um,24 h,LJP007_HT29_24H_X1_B20:P14|LJP007_HT29_24H_X2_...
19693,LJP007_HT29_24H:P15,BRD-K77625799,vandetanib,trt_cp,HT29,1.11 um,24 h,LJP007_HT29_24H_X1_B20:P15|LJP007_HT29_24H_X2_...
19694,LJP007_HT29_24H:P16,BRD-K77625799,vandetanib,trt_cp,HT29,0.37 um,24 h,LJP007_HT29_24H_X1_B20:P16|LJP007_HT29_24H_X2_...
19695,LJP007_HT29_24H:P17,BRD-K77625799,vandetanib,trt_cp,HT29,0.12 um,24 h,LJP007_HT29_24H_X1_B20:P17|LJP007_HT29_24H_X2_...
19696,LJP007_HT29_24H:P18,BRD-K77625799,vandetanib,trt_cp,HT29,0.04 um,24 h,LJP007_HT29_24H_X1_B20:P18|LJP007_HT29_24H_X2_...
104639,REP.A025_HT29_24H:K19,BRD-K77625799,vandetanib,trt_cp,HT29,10.0 um,24 h,REP.A025_HT29_24H_X1.L2_B27:K19|REP.A025_HT29_...
104640,REP.A025_HT29_24H:K20,BRD-K77625799,vandetanib,trt_cp,HT29,3.33 um,24 h,REP.A025_HT29_24H_X1.L2_B27:K20|REP.A025_HT29_...
104641,REP.A025_HT29_24H:K21,BRD-K77625799,vandetanib,trt_cp,HT29,1.11 um,24 h,REP.A025_HT29_24H_X1.L2_B27:K21|REP.A025_HT29_...
104642,REP.A025_HT29_24H:K22,BRD-K77625799,vandetanib,trt_cp,HT29,0.37 um,24 h,REP.A025_HT29_24H_X1.L2_B27:K22|REP.A025_HT29_...


con esto ahora vamos a quitar los duplicados en los que no haya firma génica de  alguna de las 2 drogas

In [38]:
# Crear un diccionario para acceso rápido: (drug, cell_line) -> num_signatures
signature_lookup = {
    (row["drug"], row["cell_line"]): row["num_signatures"]
    for _, row in signatures_df.iterrows()
}


In [39]:
def is_valid_row(row, signature_lookup):
    drug1 = row['cmaps_name_row']
    drug2 = row['cmaps_name_col']
    cell = row['cell_line_name']
    
    sig1 = signature_lookup.get((drug1, cell), 0)
    sig2 = signature_lookup.get((drug2, cell), 0)
    
    return sig1 > 0 and sig2 > 0

# Aplicar filtro
filtered_df = summary_drugcomb[summary_drugcomb.apply(lambda row: is_valid_row(row, signature_lookup), axis=1)]


In [40]:
filtered_df

,block_id,drug_row_id,drug_col_id,cell_line_id,conc_r_unit,conc_c_unit,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,...,smiles_col,molecular_formula_col,drugbank_id_col,kegg_id_col,cmaps_name_col,cell_line_name,cell_disease_id,tumor_location_cosmic,tumor_location_cellosaurus,disease_cui
0,4641,1,2,4,uM,uM,-1.832288,-26.978617,-8.166073,-4.944960,...,CC1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N,C13H16N4O,DB07232,D09692,veliparib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
1,4645,1,3,4,uM,uM,2.612379,-2.542583,5.443890,2.058551,...,CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)N...,C27H32N8O2,DB11740,NaN,MK-1775,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
2,4649,1,4,4,uM,uM,3.866124,0.871119,6.244486,2.778483,...,CC(C)(C#N)C1=CC=C(C=C1)N2C3=C4C=C(C=CC4=NC=C3N...,C30H23N5O,DB11651,D10552,NVP-BEZ235,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
3,4653,1,5,4,uM,uM,0.589177,-16.482204,-4.214315,-4.724869,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
4,4657,1,8,4,uM,uM,-0.756504,-5.749643,0.124410,-3.229846,...,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...,C22H26ClN7O2S,DB01254,D03658,dasatinib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,430657,115,1,38,uM,uM,2.007334,4.869263,6.344526,3.422500,...,C1=C(C(=O)NC(=O)N1)F,C4H3FN2O2,DB00544,D00584,fluorouracil,MCF7,C4194,Breast (Carcinoma),Invasive breast carcinoma of no special type,C1134719
4087,431615,88,5,38,uM,uM,1.339214,0.900087,1.432115,1.710700,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,MCF7,C4194,Breast (Carcinoma),Invasive breast carcinoma of no special type,C1134719
4090,431640,88,5,28,uM,uM,1.894596,5.360176,3.566226,4.395462,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,HT29,C4349,Large intestine; Colon (Carcinoma),Colon adenocarcinoma,C0338106
4091,432386,72,1,38,uM,uM,0.799656,5.414422,6.211970,1.212909,...,C1=C(C(=O)NC(=O)N1)F,C4H3FN2O2,DB00544,D00584,fluorouracil,MCF7,C4194,Breast (Carcinoma),Invasive breast carcinoma of no special type,C1134719


In [41]:
filtered_df.columns = [
    col.replace('_row', '_drug1').replace('_col', '_drug2')
    for col in filtered_df.columns
]


In [43]:
filtered_df

,block_id,drug_drug1_id,drug_drug2_id,cell_line_id,conc_r_unit,conc_c_unit,synergy_zip,synergy_loewe,synergy_hsa,synergy_bliss,...,smiles_drug2,molecular_formula_drug2,drugbank_id_drug2,kegg_id_drug2,cmaps_name_drug2,cell_line_name,cell_disease_id,tumor_location_cosmic,tumor_location_cellosaurus,disease_cui
0,4641,1,2,4,uM,uM,-1.832288,-26.978617,-8.166073,-4.944960,...,CC1(CCCN1)C2=NC3=C(C=CC=C3N2)C(=O)N,C13H16N4O,DB07232,D09692,veliparib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
1,4645,1,3,4,uM,uM,2.612379,-2.542583,5.443890,2.058551,...,CC(C)(C1=NC(=CC=C1)N2C3=NC(=NC=C3C(=O)N2CC=C)N...,C27H32N8O2,DB11740,NaN,MK-1775,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
2,4649,1,4,4,uM,uM,3.866124,0.871119,6.244486,2.778483,...,CC(C)(C#N)C1=CC=C(C=C1)N2C3=C4C=C(C=CC4=NC=C3N...,C30H23N5O,DB11651,D10552,NVP-BEZ235,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
3,4653,1,5,4,uM,uM,0.589177,-16.482204,-4.214315,-4.724869,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
4,4657,1,8,4,uM,uM,-0.756504,-5.749643,0.124410,-3.229846,...,CC1=C(C(=CC=C1)Cl)NC(=O)C2=CN=C(S2)NC3=NC(=NC(...,C22H26ClN7O2S,DB01254,D03658,dasatinib,A375,C3802,Skin (Malignant melanoma),Amelanotic melanoma,C0206735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4086,430657,115,1,38,uM,uM,2.007334,4.869263,6.344526,3.422500,...,C1=C(C(=O)NC(=O)N1)F,C4H3FN2O2,DB00544,D00584,fluorouracil,MCF7,C4194,Breast (Carcinoma),Invasive breast carcinoma of no special type,C1134719
4087,431615,88,5,38,uM,uM,1.339214,0.900087,1.432115,1.710700,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,MCF7,C4194,Breast (Carcinoma),Invasive breast carcinoma of no special type,C1134719
4090,431640,88,5,28,uM,uM,1.894596,5.360176,3.566226,4.395462,...,B(C(CC(C)C)NC(=O)C(CC1=CC=CC=C1)NC(=O)C2=NC=CN...,C19H25BN4O4,DB00188,D03150,bortezomib,HT29,C4349,Large intestine; Colon (Carcinoma),Colon adenocarcinoma,C0338106
4091,432386,72,1,38,uM,uM,0.799656,5.414422,6.211970,1.212909,...,C1=C(C(=O)NC(=O)N1)F,C4H3FN2O2,DB00544,D00584,fluorouracil,MCF7,C4194,Breast (Carcinoma),Invasive breast carcinoma of no special type,C1134719


In [42]:
filtered_df.columns

Index(['block_id', 'drug_drug1_id', 'drug_drug2_id', 'cell_line_id',
       'conc_r_unit', 'conc_c_unit', 'synergy_zip', 'synergy_loewe',
       'synergy_hsa', 'synergy_bliss', 'ic50_drug1', 'ic50_drug2', 'ri_drug1',
       'ri_drug2', 'css_drug1', 'css_drug2', 'css_ri', 'S_sum', 'quality',
       'S_mean', 'S_max', 'dname_drug1', 'chembl_id_drug1', 'inchikey_drug1',
       'smiles_drug1', 'molecular_formula_drug1', 'drugbank_id_drug1',
       'kegg_id_drug1', 'cmaps_name_drug1', 'dname_drug2', 'chembl_id_drug2',
       'inchikey_drug2', 'smiles_drug2', 'molecular_formula_drug2',
       'drugbank_id_drug2', 'kegg_id_drug2', 'cmaps_name_drug2',
       'cell_line_name', 'cell_disease_id', 'tumor_location_cosmic',
       'tumor_location_cellosaurus', 'disease_cui'],
      dtype='object')

In [44]:
filtered_df.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\combinations_cmaps_drugcomb_final.csv", index=False)

In [53]:
reduced_summary = filtered_df[['dname_drug1', 'chembl_id_drug1', 'dname_drug2', 'chembl_id_drug2',
      'cell_line_name', 'cell_disease_id','disease_cui']]

In [54]:
reduced_summary = reduced_summary.rename(columns={'cell_disease_id': 'NCI_thesaurus_code'})

In [55]:
reduced_summary

,dname_drug1,chembl_id_drug1,dname_drug2,chembl_id_drug2,cell_line_name,NCI_thesaurus_code,disease_cui
0,5-Fluorouracil,CHEMBL185,Veliparib,CHEMBL506871,A375,C3802,C0206735
1,5-Fluorouracil,CHEMBL185,MK-1775,CHEMBL1976040,A375,C3802,C0206735
2,5-Fluorouracil,CHEMBL185,915019-65-7,CHEMBL1879463,A375,C3802,C0206735
3,5-Fluorouracil,CHEMBL185,Bortezomib,CHEMBL325041,A375,C3802,C0206735
4,5-Fluorouracil,CHEMBL185,Dasatinib,CHEMBL1421,A375,C3802,C0206735
...,...,...,...,...,...,...,...
4086,Vandetanib,CHEMBL24828,5-Fluorouracil,CHEMBL185,MCF7,C4194,C1134719
4087,mitoxantrone,CHEMBL58,Bortezomib,CHEMBL325041,MCF7,C4194,C1134719
4090,mitoxantrone,CHEMBL58,Bortezomib,CHEMBL325041,HT29,C4349,C0338106
4091,Gefitinib,CHEMBL939,5-Fluorouracil,CHEMBL185,MCF7,C4194,C1134719


In [56]:
reduced_summary['disease_cui'].unique()

array(['C0206735', 'C0338106', 'C0600139', 'C1134719', 'C0152013'],
      dtype=object)

In [58]:
filtered_df.to_csv(r"C:\Users\Usuario\Desktop\TFM\datos\CLUE_data\CMAPS\combinations_cmaps_drugcomb_final_reduced.csv", index=False)